# Building ANN

In [1]:
%config IPCompleter.greedy=True


### Steps for building ANN

- Data preprocessing
- Add input layer
- Random w init
- Add hidden layers
- Select optimizer, Loss and Performance metrics
- Compile the model
- Use model.fit to train the model
- Evaluate the model
- Adjust optimization parameters or model if needed

In [2]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#### Read dataset from csv file

In [4]:
dataset = pd.read_csv('customer-data.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### x drops the following labels (col entries) while y only takes the 'Exited" values

In [5]:
x = dataset.drop(labels = ['CustomerId', 'Surname', 'RowNumber', 'Exited'], axis = 1)
y = dataset['Exited']
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [6]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

#### Map non-numerical fields using label encoder

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
geoLabel = LabelEncoder()
x['Geography'] = geoLabel.fit_transform(x['Geography'])
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,Female,42,2,0.00,1,1,1,101348.88
1,608,2,Female,41,1,83807.86,1,0,1,112542.58
2,502,0,Female,42,8,159660.80,3,1,0,113931.57
3,699,0,Female,39,1,0.00,2,0,0,93826.63
4,850,2,Female,43,2,125510.82,1,1,1,79084.10


In [9]:
genderLabel = LabelEncoder()
x['Gender'] = genderLabel.fit_transform(x['Gender'])
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


#### Use one hot encoding instead

In [10]:
x = pd.get_dummies(x, drop_first = True, columns=['Geography'])
x.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_1,Geography_2
0,619,0,42,2,0.00,1,1,1,101348.88,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,1


#### Some data vary too much, implement feature standarlization (scale it)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0, stratify = y)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [13]:
x_train

array([[-1.24021723, -1.09665089,  0.77986083, ...,  1.64099027,
        -0.57812007, -0.57504086],
       [ 0.75974873,  0.91186722, -0.27382717, ..., -1.55587522,
         1.72974448, -0.57504086],
       [-1.72725557, -1.09665089, -0.9443559 , ...,  1.1038111 ,
        -0.57812007, -0.57504086],
       ...,
       [-0.51484098,  0.91186722,  0.87565065, ..., -1.01507508,
         1.72974448, -0.57504086],
       [ 0.73902369, -1.09665089, -0.36961699, ..., -1.47887193,
        -0.57812007, -0.57504086],
       [ 0.95663657,  0.91186722, -1.32751517, ...,  0.50945854,
        -0.57812007,  1.73900686]])

In [14]:
x_test

array([[-1.975956  , -1.09665089,  0.1093321 , ..., -0.15254674,
         1.72974448, -0.57504086],
       [ 0.64576103, -1.09665089, -1.42330499, ...,  0.36818473,
         1.72974448, -0.57504086],
       [ 0.12763513,  0.91186722,  2.40828774, ..., -1.46485588,
        -0.57812007,  1.73900686],
       ...,
       [-1.20912968,  0.91186722,  0.20512192, ..., -1.49962429,
        -0.57812007, -0.57504086],
       [ 1.10171182, -1.09665089,  1.54617938, ..., -0.10699919,
        -0.57812007,  1.73900686],
       [ 0.9773616 , -1.09665089, -0.75277626, ...,  0.69518685,
        -0.57812007,  1.73900686]])

#### Build ANN

In [15]:
model = Sequential()

In [16]:
model.add(Dense(x.shape[1], activation = 'relu', input_dim = x.shape[1]))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))

In [17]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
model.fit(x_train, y_train.to_numpy(), batch_size = 5, epochs = 10, verbose = 1)

Train on 8000 samples
Epoch 1/10
8000/8000 [==============================] - 5s 624us/sample - loss: 0.4368 - accuracy: 0.8146
Epoch 2/10
8000/8000 [==============================] - 4s 447us/sample - loss: 0.3704 - accuracy: 0.8471
Epoch 3/10
8000/8000 [==============================] - 4s 446us/sample - loss: 0.3533 - accuracy: 0.8574
Epoch 4/10
8000/8000 [==============================] - 4s 446us/sample - loss: 0.3454 - accuracy: 0.8584
Epoch 5/10
8000/8000 [==============================] - 4s 447us/sample - loss: 0.3421 - accuracy: 0.8593
Epoch 6/10
8000/8000 [==============================] - 4s 447us/sample - loss: 0.3392 - accuracy: 0.8596
Epoch 7/10
8000/8000 [==============================] - 4s 447us/sample - loss: 0.3368 - accuracy: 0.8608
Epoch 8/10
8000/8000 [==============================] - 4s 446us/sample - loss: 0.3343 - accuracy: 0.8631
Epoch 9/10
8000/8000 [==============================] - 4s 448us/sample - loss: 0.3322 - accuracy: 0.8622
Epoch 10/10
8000/8000 [=

#### Predictions

In [19]:
y_pred = model.predict_classes(x_test)
model.evaluate(x_test, y_test.to_numpy())

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.34616838991641996, 0.8575]

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [21]:
confusion_matrix(y_test, y_pred)

array([[1536,   57],
       [ 228,  179]], dtype=int64)

In [22]:
accuracy_score(y_test, y_pred)

0.8575